# 0 - Environment Set Up 

In [ ]:
# Install required packages
!pip install -r ../requirements.txt

# 1 - Project Set Up

## 1.1 - Map to Root

In [ ]:
REL_PATH_TO_ROOT = "../"

In [ ]:
import sys
import os
import json
import pandas as pd

In [ ]:
sys.path.insert(0,REL_PATH_TO_ROOT)

In [ ]:
from src.utils import get_root_dir, test_root_dir
from local_variables import ROOT_DIR

In [ ]:
test_root_dir(REL_PATH_TO_ROOT)

## 1.2 - Set-Up Imports

In [ ]:
from prompt_manager.manager import PromptManager
from prompt_manager.fetcher import fetch_prompt
from src.api import generate_outputs_openai

## 1.3 - Load Hallucination Dataset

In [ ]:
input_path = "../data/hallucination_qa_data.json"

with open(input_path, 'r', encoding="utf-8") as f:
    data = []
    for line in f:
        data.append(json.loads(line))

    hallucination_df = pd.DataFrame(data)
            

In [ ]:
hallucination_df.shape

In [ ]:
hallucination_df.head()

## 1.4 - Sample from the Hallucination Dataset

The original dataset contains 10K records. To lower the likelihood of hitting rate limit, we need to created a random sample from the original data. Sugguested size: 100-150 records.

# 2 - Run Your Evaluation 

## 2.1 - Get Your Prompts

In [ ]:
SEQUENCE = ["task_1","hallucination_detector"]

prompt_template = fetch_prompt(SEQUENCE,use_latest_version=True)

print(prompt_template)

In [ ]:
evaluator_responses = []

# Loop through dataset
for _, row in hal_df.iterrows():

    # Get inputs and place into dictionary format
    context = row["knowledge"]

    response = row["chatbot_response"]

    row_inputs = {"CONTEXT" : context, "RESPONSE" : response}

    # Initialise prompt to validate and format inputs
    prompt = PromptManager(template=prompt_template,inputs=row_inputs)
    prompt.validate_inputs()
    prompt.format_inputs()

    # Send prompt and collect response
    response = generate_outputs_openai(prompt.prompt)
    evaluator_responses.append(response)

hallucination_df["evaluator_response"] = evaluator_responses
display(hallucination_df.head(5))